In [1]:
# -*- Encoding: UTF-8 -*-
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

'''
각 단어별로 결과를 찾아낼 수 있도록 합니다.
--------
h -> o
e -> l
l -> l
l -> e
o -> h
--------
같은 l이지만 그 결과가 l 혹은 e로 출력이 되어야 할 때가 있습니다.
'''

idx2char = ['h', 'e', 'l', 'o'] # 등장하는 글자 명시하기

'''
h: 0, e: 1, l: 2, o: 3
'''
x_data = [[0, 1, 2, 2, 3]]  # hello: 입력
x_one_hot = [[[1, 0, 0, 0], # h 0
              [0, 1, 0, 0], # e 1
              [0, 0, 1, 0], # l 2
              [0, 0, 1, 0], # l 2
              [0, 0, 0, 1]]] # o 3

y_data = [[3, 2, 2, 1, 0]] # olleh: 학습 목표

num_classes = 5
input_dim = 4  # one-hot size (글자 종류: 4개)
hidden_size = 5 # (출력 개수: 5개)
batch_size = 1   # one sentence
sequence_length = 5  # 글자 길이: 5개
learning_rate = 0.1

# X는 one-hot
X = tf.placeholder(
    tf.float32, [None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, [None, sequence_length])
             
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

In [2]:
# FC Layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(
    inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])

'''
예측: logits=outputs
목표: targets=Y
가중치: [1, 1, 1, 1, 1]
'''
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)

# Gradient 대신 성능이 좋은 Adam을 사용합니다.
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

In [3]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.67305 prediction:  [[1 1 1 1 1]] true Y:  [[3, 2, 2, 1, 0]]
	Prediction str:  eeeee
1 loss: 1.54734 prediction:  [[2 2 1 1 1]] true Y:  [[3, 2, 2, 1, 0]]
	Prediction str:  lleee
2 loss: 1.46033 prediction:  [[2 2 2 1 1]] true Y:  [[3, 2, 2, 1, 0]]
	Prediction str:  lllee
3 loss: 1.35581 prediction:  [[2 2 2 1 0]] true Y:  [[3, 2, 2, 1, 0]]
	Prediction str:  llleh
4 loss: 1.23358 prediction:  [[2 2 2 0 0]] true Y:  [[3, 2, 2, 1, 0]]
	Prediction str:  lllhh
5 loss: 1.10949 prediction:  [[2 2 2 0 0]] true Y:  [[3, 2, 2, 1, 0]]
	Prediction str:  lllhh
6 loss: 0.983369 prediction:  [[2 2 2 1 0]] true Y:  [[3, 2, 2, 1, 0]]
	Prediction str:  llleh
7 loss: 0.847668 prediction:  [[2 2 2 1 0]] true Y:  [[3, 2, 2, 1, 0]]
	Prediction str:  llleh
8 loss: 0.70917 prediction:  [[3 2 2 1 0]] true Y:  [[3, 2, 2, 1, 0]]
	Prediction str:  olleh
9 loss: 0.586351 prediction:  [[3 2 2 1 0]] true Y:  [[3, 2, 2, 1, 0]]
	Prediction str:  olleh
10 loss: 0.482305 prediction:  [[3 2 2 1 0]] true Y:  [[3